In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel,AutoTokenizer,Trainer,TrainingArguments,GPT2Config
from torch.utils.data.dataloader import DataLoader, Dataset
import os
import tqdm
from random import randint
import glob

In [2]:
files_name =[file for file in glob.glob("../input/tho-5-chu/5 chu/*.txt")]
len(files_name)

6747

In [3]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
tokenizer.add_tokens('\n')

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

In [4]:
tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

1

In [5]:
tokenizer.convert_tokens_to_ids(tokenizer.bos_token)

0

In [6]:
tokenizer.convert_tokens_to_ids(tokenizer.eos_token)

2

In [7]:
class CustomerDataset(Dataset):
    def __init__(self,tokenizer,file_path,block_size: int):
        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)
#         print(f"block_size = {block_size}")
        self.examples = []
        self.mask = []
        for file in file_path:
            with open(file, encoding="utf-8") as f:
                text = f.read()
            # text --> token-->number
            tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
#             print(tokenized_text)
#             while i < len(tokenized_text) - block_size + 1:
#                 inds = tokenized_text[i : i + block_size]
            if len(tokenized_text) < block_size:
                inds = [tokenizer.convert_tokens_to_ids(tokenizer.bos_token)] + tokenized_text + [tokenizer.convert_tokens_to_ids(tokenizer.eos_token)] + \
            (block_size - len(tokenized_text)) * [tokenizer.convert_tokens_to_ids(tokenizer.pad_token)]
                mask = [x != tokenizer.convert_tokens_to_ids(tokenizer.pad_token) for x in inds]
#                 print(mask)
            else:
                inds = [tokenizer.convert_tokens_to_ids(tokenizer.bos_token)] + tokenized_text[:block_size] + [tokenizer.convert_tokens_to_ids(tokenizer.eos_token)]
                mask = [1] * len(inds)
#                 print(mask)
#             print(inds)
#             for j in range(0, len(inds)):
#                 if inds[j]==64001:
#                     inds = inds[j+1:] #remove the first \n
#                     break
#             for j in range(len(inds)-1, 0, -1):
#                 if inds[j]==64001:
#                     inds = inds[:j-1] #remove \n
#                     break
#             print(inds)
#             print(f"len inds is {len(inds)}")
#             self.examples.append(tokenizer.build_inputs_with_special_tokens(inds))
            self.examples.append(inds) 
            self.mask.append(mask)
            
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return {
            "input_ids":torch.tensor(self.examples[i], dtype=torch.long), 
            "attention_mask":torch.tensor(self.mask[i], dtype = torch.long)
        }


In [8]:

from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling, LineByLineWithSOPTextDataset

def load_dataset(train_path, tokenizer):
    train_dataset = CustomerDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size= 256)#256
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,data_collator

In [9]:
train_dataset,data_collator = load_dataset(files_name,tokenizer)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
model = GPT2LMHeadModel.from_pretrained('danghuy1999/gpt2-viwiki')
rand_weight = torch.rand(model.lm_head.weight.shape)
model.lm_head.weight = torch.nn.parameter.Parameter(rand_weight)
task_gpt2 = {"text-generation": {"do_sample": True, "max_length": 256}} 
configuration = GPT2Config(vocab_size=64002, n_positions=260, n_ctx=260,
                           task_specific_params=task_gpt2,
                           eos_token_id = 2,
                           bos_token_id = 0,
                           pad_token_id = 1,
                           sep_token_id = 2,)
model = GPT2LMHeadModel(configuration)
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

Downloading:   0%|          | 0.00/916 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

Some weights of the model checkpoint at danghuy1999/gpt2-viwiki were not used when initializing GPT2LMHeadModel: ['multiple_choice_head.summary.bias', 'multiple_choice_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
model = model.from_pretrained('../input/5chusaved')

In [13]:
from transformers.trainer_callback import TrainerCallback
from transformers import pipeline

training_args = TrainingArguments(
    output_dir="5-chu", 
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16, 
    save_steps=1000,
    save_total_limit = 2,
    warmup_steps=1000, 
    logging_steps=100,
    report_to="wandb"
    )

In [14]:
device = torch.device('cuda')
trainer = Trainer(
    model=model, 
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [15]:
trainer.train()


***** Running training *****
  Num examples = 6747
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16880
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Step,Training Loss
100,0.331600
200,0.329900
300,0.334500
400,0.345700
500,0.361900
600,0.387000
700,0.413900
800,0.442900
900,0.453900
1000,0.478000


Saving model checkpoint to 5-chu/checkpoint-1000
Configuration saved in 5-chu/checkpoint-1000/config.json
Model weights saved in 5-chu/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to 5-chu/checkpoint-2000
Configuration saved in 5-chu/checkpoint-2000/config.json
Model weights saved in 5-chu/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to 5-chu/checkpoint-3000
Configuration saved in 5-chu/checkpoint-3000/config.json
Model weights saved in 5-chu/checkpoint-3000/pytorch_model.bin
Deleting older checkpoint [5-chu/checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to 5-chu/checkpoint-4000
Configuration saved in 5-chu/checkpoint-4000/config.json
Model weights saved in 5-chu/checkpoint-4000/pytorch_model.bin
Deleting older checkpoint [5-chu/checkpoint-2000] due to args.save_total_limit
Saving model checkpoint to 5-chu/checkpoint-5000
Configuration saved in 5-chu/checkpoint-5000/config.json
Model weights saved in 5-chu/checkpoint-5000/pytorch_model.b

TrainOutput(global_step=16880, training_loss=0.25742239759996605, metrics={'train_runtime': 6007.7006, 'train_samples_per_second': 22.461, 'train_steps_per_second': 2.81, 'total_flos': 1.762937339904e+16, 'train_loss': 0.25742239759996605, 'epoch': 20.0})

In [16]:
ls 5-chu/

checkpoint-15000/  checkpoint-16000/


In [17]:

from transformers import pipeline
poem = pipeline('text-generation', model="./5-chu/checkpoint-16000/", tokenizer=tokenizer)
#Test


loading configuration file ./5-chu/checkpoint-16000/config.json
Model config GPT2Config {
  "_name_or_path": "../input/5chusaved",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 260,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 260,
  "pad_token_id": 1,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "sep_token_id": 2,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 256
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "

In [18]:
a = poem('<s>mẹ ơi')
print(a[0]['generated_text'])

<s>mẹ ơi con rất sợ 
 mỗi khi mùa xuân về 
 tóc mẹ lại thêm bạc 
 lưng mẹ cũng thêm còng 
 mẹ ơi con rất sợ 
 mỗi năm mùa xuân về 
 mẹ ngồi ngóng con trẻ 
 có đứa nào về chưa 
 mẹ ơi con vẫn sợ 
 nhưng mùa xuân vần về 
 mẹ lại thêm tuổi mới 
 chuối chín cây sắp rơi 
 mẹ ơi con vẫn sợ 
 rồi một khi xuân về 
 không còn thấy bóng mẹ 
 trước cổng ngồi ngóng con
